<a href="https://colab.research.google.com/github/ashikita/research_data_title/blob/main/get_research_data_title.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import csv
import time

# -----------------------------
# 設定パラメータ（変更可能）
# -----------------------------
affiliation_id = "00p4k0j84"       # 九州大学 ROR ID
resource_type_id = "dataset"       # データセットのみ
has_citations = "1"                # 引用あり
include_affiliation = "true"       # 所属情報を含める
page_size = 100                    # 1ページあたりの件数
output_file = "output.tsv"         # 出力ファイル名

# ベースURLの構築
base_url = (
    f"https://api.datacite.org/dois?"
    f"affiliation-id={affiliation_id}"
    f"&resource-type-id={resource_type_id}"
    f"&has-citations={has_citations}"
    f"&affiliation={include_affiliation}"
    f"&page[size]={page_size}"
)

# 出力ファイルの準備
total_count = 0
start_time = time.time()

with open(output_file, "w", newline='', encoding="utf-8") as file:
    writer = csv.writer(file, delimiter='\t')
    writer.writerow(["doi", "title", "publisher", "publicationYear"])

    url = base_url
    while url:
        response = requests.get(url)
        if response.status_code != 200:
            print(f"データ取得失敗: {response.status_code}")
            break

        data = response.json()

        # 各レコードから必要な情報を抽出
        for item in data.get("data", []):
            attr = item.get("attributes", {})
            doi = attr.get("doi", "")
            titles = attr.get("titles", [])
            title = titles[0].get("title", "") if titles else ""
            publisher = attr.get("publisher", "")
            year = attr.get("publicationYear", "")
            writer.writerow([doi, title, publisher, year])
            total_count += 1

        # 次ページがあるか確認
        url = data.get("links", {}).get("next")

# 経過時間と件数の表示
elapsed_time = time.time() - start_time
print(f"✅ 完了: {total_count} 件のデータを取得しました。")
print(f"⏱️ 経過時間: {elapsed_time:.2f} 秒")